---
title: Observe OpenAI Structured Outputs with Langfuse
description: Learn how to use Langfuse to monitor OpenAI Structured Outputs
category: Integrations
---

# Cookbook: Trace OpenAI Structured Outputs with Langfuse

In this cookbook you will learn how to use Langfuse to monitor OpenAI Structured Outputs.

## What are structured outputs?
Generating structured data from unstructured inputs is a core AI use case today. Structured outputs make especially chained LLM calls, UI component generation, and model-based evaluation more reliable. [Structured Outputs](https://openai.com/index/introducing-structured-outputs-in-the-api/) is a new capability of the OpenAI API that builds upon JSON mode and function calling to enforce a strict schema in a model output.

## How to trace structured output in Langfuse?
If you use the OpenAI Python SDK, you can use the [Langfuse drop-in replacement](https://langfuse.com/docs/integrations/openai/python/get-started) to get full logging by changing only the import. With that, you can monitor the structured output generated by OpenAI in Langfuse.

```diff
- import openai
+ from langfuse.openai import openai

Alternative imports:
+ from langfuse.openai import OpenAI, AsyncOpenAI, AzureOpenAI, AsyncAzureOpenAI
```



## Step 1: Initialize Langfuse
Initialize the Langfuse client with your [API keys](https://langfuse.com/faq/all/where-are-langfuse-api-keys) from the project settings in the Langfuse UI and add them to your environment.

In [ ]:
%pip install langfuse openai --upgrade

In [2]:
import os

# Get keys for your project from the project settings page
# https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your openai key
os.environ["OPENAI_API_KEY"] = ""

## Step 2: Math tutor example

In this example, we'll build a math tutoring tool that outputs steps to solve a math problem as an array of structured objects.

This setup is useful for applications where each step needs to be displayed separately, allowing users to progress through the solution at their own pace.

(Example taken from [OpenAI cookbook](https://cookbook.openai.com/examples/structured_outputs_intro))

In [3]:
# Use the Langfuse drop-in replacement to get full logging by changing only the import.
# With that, you can monitor the structured output generated by OpenAI in Langfuse.
from langfuse.openai import OpenAI
import json

openai_model = "gpt-4o-2024-08-06"
client = OpenAI()

In the `response_format` parameter you can now supply a JSON Schema via `json_schema`. When using `response_format` with `strict: true`, the model's output will adhere to the provided schema.

Function calling remains similar, but with the new parameter `strict: true`, you can now ensure that the schema provided for the functions is strictly followed.

In [4]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

def get_math_solution(question):
    response = client.chat.completions.create(
    model = openai_model,
    messages=[
        {
            "role": "system",
            "content": math_tutor_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )

    return response.choices[0].message

In [5]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question)

print(result.content)

{"steps":[{"explanation":"First, we need to isolate the term with the variable, 8x, by eliminating the constant term on the left-hand side. We do this by subtracting 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"After subtracting 7 from both sides, the equation simplifies to 8x = -30.","output":"8x = -30"},{"explanation":"Next, to solve for x, we need to divide both sides of the equation by 8, the coefficient of x.","output":"8x/8 = -30/8"},{"explanation":"Dividing each side by 8 gives us x = -30/8, which simplifies to x = -15/4. This is done by dividing both the numerator and the denominator by 2.","output":"x = -15/4"}],"final_answer":"x = -15/4"}


In [6]:
# Print results step by step
from IPython.display import Math, display

def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")

    print("Final answer:\n\n")
    display(Math(final_answer))

print_math_response(result.content)

Step 1: First, we need to isolate the term with the variable, 8x, by eliminating the constant term on the left-hand side. We do this by subtracting 7 from both sides of the equation.



<IPython.core.display.Math object>



Step 2: After subtracting 7 from both sides, the equation simplifies to 8x = -30.



<IPython.core.display.Math object>



Step 3: Next, to solve for x, we need to divide both sides of the equation by 8, the coefficient of x.



<IPython.core.display.Math object>



Step 4: Dividing each side by 8 gives us x = -30/8, which simplifies to x = -15/4. This is done by dividing both the numerator and the denominator by 2.



<IPython.core.display.Math object>



Final answer:




<IPython.core.display.Math object>

## Step 3: See your trace in Langfuse

You can now see the trace and the JSON schema in Langfuse.

[Example trace in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/3ecc3849-66c9-4eaf-b26b-bde26b7eebed)

![View example trace in the Langfuse UI](https://langfuse.com/images/cookbook/integration-openai-structured-outputs-tracing.png)

## Feedback

If you have any feedback or requests, please create a GitHub [Issue](https://langfuse.com/issue) or share your idea with the community on [Discord](https://langfuse.com/discord).